In [ ]:
import time
import math
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.mlab as mlab
from scipy.linalg import qr

import keras
from keras.models import Sequential, Model, model_from_json
from keras.layers import Input, Dense, Dropout, BatchNormalization, merge, LocallyConnected1D, Flatten, Conv1D,LeakyReLU
from keras import backend as K
from keras import regularizers
#from keras.objectives import mse
from keras import regularizers, optimizers
from keras.callbacks import EarlyStopping
from keras.initializers import Constant

from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.metrics import accuracy_score,cohen_kappa_score,roc_auc_score,average_precision_score


batch_size = 64;
filterNum = 1;
bias = True;
iterNum = 1;


def show_layer_info(layer_name, layer_out):
    # print('[layer]: %s\t[shape]: %s \n' % (layer_name,str(layer_out.get_shape().as_list())))
    pass



def build_DNN(p, coeff=0):

    input = Input(name='input', shape=(p, 2));
    show_layer_info('Input', input);

    local1 = LocallyConnected1D(filterNum,1, use_bias=bias, kernel_initializer=Constant(value=0.1))(input);
    show_layer_info('LocallyConnected1D', local1);

    local2 = LocallyConnected1D(1,1, use_bias=bias, kernel_initializer='glorot_normal')(local1);
    show_layer_info('LocallyConnected1D', local2);

    flat = Flatten()(local2);
    show_layer_info('Flatten', flat);

    dense1 = Dense(int(h_size),use_bias=bias, kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l1(coeff))(flat)
    x = LeakyReLU(0.2)(dense1)

    dense2 = Dense(int(h_size),use_bias=bias, kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l1(coeff))(x)
    x = LeakyReLU(0.2)(dense2)

    dense3 = Dense(int(h_size),use_bias=bias, kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l1(coeff))(x)
    x = LeakyReLU(0.2)(dense3)
    
    out_ = Dense(1, activation='sigmoid', kernel_initializer='glorot_normal')(x)


    model = Model(inputs=input, outputs=out_)
    
    
    opt = optimizers.Adam(learning_rate=0.0005, epsilon=1e-6)
    
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model


def train_DNN(model, X, y, myCallback):
    num_sequences = len(y);
    num_positives = np.sum(y);
    num_negatives = num_sequences - num_positives;
    
     
    model.fit(X, y, epochs=num_epochs, batch_size=batch_size, verbose=1, validation_split = 0.2,class_weight={True: num_sequences / num_positives, False: num_sequences / num_negatives}, callbacks=[myCallback]);
    return model;


def test_DNN(model, X, y):
    return roc_auc_score(y, model.predict(X));

def predict_DNN(model, X):
    return model.predict(X);

def plot_roc(model, X, y):
    fpr, tpr, thresholds = roc_curve(y_true=y, y_score=model.predict(X));
    roc_auc = auc(x=fpr, y=tpr)
    plt.plot(fpr, tpr, linestyle='-', label='auc={:.4f}'.format(roc_auc));
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', linewidth=2)
    plt.legend(loc='lower right', fontsize=16)
    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.1, 1.1])
    plt.ylabel('True Positive Rate', fontsize=20)
    plt.xlabel('False Positive Rate', fontsize=20)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.show();

class My_Callback(keras.callbacks.Callback):
    def __init__(self, outputDir, pVal):
        self.outputDir = outputDir;
        self.pVal = pVal;
        print(self.outputDir);

    def on_epoch_end(self, epoch, logs={}):
        if ((epoch+1) % 100) != 0: return;

        h_local1_weight = np.array(self.model.layers[1].get_weights()[0]);
        h_local2_weight = np.array(self.model.layers[2].get_weights()[0]);

        print('h_local1_weight = ' + str(h_local1_weight.shape))
        print('h_local2_weight = ' + str(h_local2_weight.shape))
        h0 = np.zeros((self.pVal, 2));
        h0_abs = np.zeros((self.pVal, 2));

        for pIdx in range(self.pVal):
            h0[pIdx, :] = np.matmul(h_local1_weight[pIdx, :, :], h_local2_weight[pIdx, :, :]).flatten();
            h0_abs[pIdx, :] = np.matmul(np.fabs(h_local1_weight[pIdx, :, :]), np.fabs(h_local2_weight[pIdx, :, :])).flatten();

        print('h0 = ' + str(h0.shape))
        print('h0_abs = ' + str(h0_abs.shape))

        h1 = np.array(self.model.layers[4].get_weights()[0]);
        h2 = np.array(self.model.layers[6].get_weights()[0]);
        h3 = np.array(self.model.layers[8].get_weights()[0]); 
        h4 = np.array(self.model.layers[10].get_weights()[0]) 

        print('h1 = ' + str(h1.shape))
        print('h2 = ' + str(h2.shape))
        print('h3 = ' + str(h3.shape))
        print('h4 = ' + str(h3.shape)) 

        W1 = h1;
        W_curr = h1;
        W2 = np.matmul(W_curr, h2);
        W_curr = np.matmul(W_curr, h2); 
        W3 = np.matmul(W_curr, h3); 
        W_curr = np.matmul(W_curr, h3)
        W4 = np.matmul(W_curr, h4) 
  
        print('W1 = ' + str(W1.shape))
        print('W2 = ' + str(W2.shape))
        print('W3 = ' + str(W3.shape))
        v0_h0 = h0[:, 0].reshape((self.pVal, 1));
        v1_h0 = h0[:, 1].reshape((self.pVal, 1));
        v0_h0_abs = h0_abs[:, 0].reshape((self.pVal, 1));
        v1_h0_abs = h0_abs[:, 1].reshape((self.pVal, 1));

        
        v3 = np.vstack((np.sum(np.square(np.multiply(v0_h0_abs, np.fabs(W3))), axis=1).reshape((self.pVal, 1)), np.sum(np.square(np.multiply(v1_h0_abs, np.fabs(W3))), axis=1).reshape((self.pVal, 1)))).T;

        v5 = np.vstack((np.sum(np.multiply(v0_h0, W3), axis=1).reshape((self.pVal, 1)),
                        np.sum(np.multiply(v1_h0, W3), axis=1).reshape((self.pVal, 1)))).T;
        
        df = pd.DataFrame(data=v3.flatten())
        df.to_csv(self.outputDir) 




In [ ]:
def cv_deepPINK(xOr,xK,yvalues,n_split,model_name):
    
    
    xvalues = np.concatenate((xOr,xK),axis=1)
    kf = KFold(n_splits=n_split)

    
    y_preds = []
    aucc = []
    auprc = []

    pVal = int(xvalues.shape[1] / 2)
    n = xvalues.shape[0]

    X_origin = xvalues[:, :pVal]
    X_knockoff = xvalues[:, pVal:]

    
    fold = 0
    for tr_idx, te_idx in kf.split(X_origin):
        
        X_tr_o, X_te_o = X_origin[tr_idx], X_origin[te_idx]
        X_tr_k, X_te_k = X_knockoff[tr_idx], X_knockoff[te_idx]
        y_tr, y_te = yvalues[tr_idx], yvalues[te_idx]
    
        n_tr = X_tr_o.shape[0]
        n_te = X_te_o.shape[0]
        
        x3D_train = np.zeros((n_tr, pVal, 2))
        x3D_train[:, :, 0] = X_tr_o
        x3D_train[:, :, 1] = X_tr_k
        x3D_test = np.zeros((n_te, pVal, 2))
        x3D_test[:, :, 0] = X_te_o
        x3D_test[:, :, 1] = X_te_k
        label_train = y_tr
        label_test = y_te
    
        
        coeff = 0.05 * np.sqrt(2.0 * np.log(pVal) / n)
        dataDir = '/Users/utente/Documents/università/tesi - confronto FS/analisi 2-512 features'
        outputDir = os.path.join( dataDir, 'CV_' + model_name + '_fold' + str(fold))
       
        
        DNN = build_DNN(pVal, coeff)
        model = DNN

        myCallback = My_Callback(outputDir,pVal)
        DNN = train_DNN(DNN, x3D_train, label_train, myCallback)
        
        hist_df = pd.DataFrame(model.history.history)
        
        y_pred=predict_DNN(model, x3D_test)
        
        fold += 1
        
        aucc.append(roc_auc_score(label_test,y_pred))
        auprc.append(average_precision_score(label_test, y_pred))
        
        
    return np.mean(aucc), np.var(aucc),np.mean(auprc),np.var(auprc)

def set_data(path,data_name,k_feat):
    
    data = np.genfromtxt(path+ data_name + str(k_feat) + 'feat.csv',delimiter=',')

    X = data[:,:-1]
    y = data[:,-1]
    
    return X,y

def matches(best_feat,non_zero):
    #non_zero = non_zeros.copy()
    match = []
    for i in range(len(best_feat)):
        for j in range(len(non_zero)):
            if best_feat[i] == non_zero[j]:
                match.append(best_feat[i])
    umatch = []
    sorted_match = np.sort(match)
    for i in range(len(sorted_match)):
        if i == len(sorted_match)-1:
        
            umatch.append(sorted_match[i])
        
        if  i < len(sorted_match)-1 and sorted_match[i] < sorted_match[i+1] :
        
            umatch.append(sorted_match[i])
        
    return umatch#,match

def get_features(model_names,n_split,k):                     
    
    best_feat = np.zeros((n_split,k))
                     
    
        
    for i in range(n_split):
    
        file = './DeepPINK/CV_' + model_names[1] + '_fold' + str(i) 
        imp = np.genfromtxt(file,delimiter=',')
        imp = imp[1:,1]
            
        p = int(imp.shape[0]/2)
           
        importances = imp[:p]-imp[p:]
                     
                     
        best_feat[i] = importances.argsort()[-k:][::-1]
        
    return best_feat

In [ ]:
dataDir = '/Users/utente/Documents/università/tesi - confronto FS/analisi 2-512 features'

path='./data/'
data_name='ring-xor-sum_1000samples-'
DK_or_2o = '_DK'
h_size = 64
num_epochs = 1000
splits = 6
comments = '_3layers64'
dataset = 'ring-xor-sum'
tot_feats = [6,8,16,32,64,128,256,512]

cv_auc = []
var_auc = []
cv_auprc = []
var_auprc = []

for i in tot_feats:
    
    # import original data and knockoff features
    X_original,y = set_data(path,data_name,i)
    X_2o = np.loadtxt(dataDir+'/data/2o_knockoff.csv',delimiter=',')
    X_DK = np.loadtxt(dataDir+'/data/DK_knockoff.csv',delimiter=',')
    
    # multiply input by 100 (I obtained better results in this way)
    X_original = X_original*100
    X_2o = X_2o[:1000,:i]*100
    X_DK = X_DK[:1000,:i]*100

    if DK_or_2o == '_DK':
        X_knock = X_DK
    else:
        X_knock = X_2o
    
    auccc,vauccc,auprc,vauprc = cv_deepPINK(X_original,X_knock,y,splits,dataset+'_'+str(i)+DK_or_2o+comments)

    var_auc.append(sauccc)
    cv_auc.append(auccc)
    var_auprc.append(sauprc)
    cv_auprc.append(auprc)

In [ ]:
splits = 6
K_feat = 6
name0 = 'DeepPINK'
dataset = 'ring-xor-sum_'

feat_res = []
feat_res_2k = []


# Best K features

for i in tot_feats:
    feat_res.append(get_features([name0,dataset+str(i)+DK_or_2o+comments],splits,K_feat))
    
m = np.zeros((len(tot_feats),splits)) #rows=ring2,ring4,ring8... columns=fold0,fold1,fold2...
for j in range(len(tot_feats)):
    for k in range(splits):
        m[j,k] =len(matches(feat_res[j][k],np.arange(K_feat)))

# best K feat averaged on the 6 folds
cv_feat = np.mean(m,axis=1) 
var_feat = np.var(m,axis=1)

# Best 2K features

for i in tot_feats:
    if i == 6 or i == 8:
        feat_res_2k.append(get_features([name0,dataset+str(i)+name1],splits,int(K_feat)))
    else:
        feat_res_2k.append(get_features([name0,dataset+str(i)+name1],splits,int(2*K_feat)))

m = np.zeros((len(tot_feats),splits)) #rows=ring2,ring4,ring8... columns=fold0,fold1,fold2...
for j in range(len(tot_feats)):
    for k in range(splits):
        
        m[j,k] =len(matches(feat_res_2k[j][k],np.arange(int(K_feat))))
        
# best 2K feat averaged on the 6 folds
cv_feat_2k = np.mean(m,axis=1)
var_feat_2k = np.var(m,axis=1)

In [ ]:
# Save results

df = pd.DataFrame([cv_auc,var_auc,cv_auprc,var_auprc,cv_feat,var_feat,cv_feat_2k,var_feat_2k])
df1 = df.T
df1
df1.to_excel("Auc-Auprc-KFeat-2KFeat_DeepPINK"+DK_or_2o+"_"+dataset[:-1]+".xlsx")